In [21]:
import pygame
from pygame.locals import *
import random
import numpy as np
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

import sys
sys.path.insert(0, '../')
from src.time_tagger import TT_Simulator, two_particle_states

In [22]:
SimulatorPhiPlus = TT_Simulator(two_particle_states['phi_plus'], initial_state_noise=0, debug=False)
CHSH_angles = SimulatorPhiPlus.CHSH_angles_for_filters

theta_a = np.random.choice(CHSH_angles[0:2])
theta_b = np.random.choice(CHSH_angles[2:4])

theta_a = 0
theta_b = 0

outcomeProbabilities = SimulatorPhiPlus.outcome_probabilities(theta_a, theta_b)[:,0]
print(outcomeProbabilities)

[0.5 0.  0.  0.5]


In [24]:
# Initialize Pygame
pygame.init()

# Set up display for 1080p windowed mode
display = (500, 400)
pygame.display.set_mode(display, DOUBLEBUF | OPENGL)

# Adjust the orthographic projection to account for display size
gluOrtho2D(0, display[0], 0, display[1])

# Initial positions for the random walks
walks = [
    {
        "x": display[0] // 4,
        "y": display[1] // 2,
        "path": [(display[0] // 4, display[1] // 2)],
        "color": (1.0, 0.0, 0.0),  # Red
        "path_color": (1.0, 0.5, 0.5)  # Light Red
    },
    {
        "x": 3 * display[0] // 4,
        "y": display[1] // 2,
        "path": [(3 * display[0] // 4, display[1] // 2)],
        "color": (0.0, 1.0, 0.0),  # Green
        "path_color": (0.5, 1.0, 0.5)  # Light Green
    }
]

# Initialize the third point
midpoint = {
    "x": (walks[0]["x"] + walks[1]["x"]) // 2,
    "y": (walks[0]["y"] + walks[1]["y"]) // 2,
    "path": [((walks[0]["x"] + walks[1]["x"]) // 2, (walks[0]["y"] + walks[1]["y"]) // 2)],
    "color": (0.0, 0.0, 1.0),  # Blue
    "path_color": (0.5, 0.5, 1.0)  # Light Blue
}

# Function to perform a random walk step
def random_walk_step(walker, walk):
    direction = np.random.choice(a=['HH', 'HV', 'VH', 'VV'], p=outcomeProbabilities)
    if direction[walker] == 'H':
        walk['x'] += np.random.choice([1, -1])
    elif direction[walker] == 'V':
        walk['y'] += np.random.choice([1, -1])

    walk['path'].append((walk['x'], walk['y']))

# Function to update the midpoint
def update_midpoint(midpoint, walk1, walk2):
    midpoint["x"] = (walk1["x"] + walk2["x"]) // 2
    midpoint["y"] = (walk1["y"] + walk2["y"]) // 2
    midpoint["path"].append((midpoint["x"], midpoint["y"]))

# Function to draw the walk path
def draw_path(walk):
    glBegin(GL_LINES)
    glColor3f(*walk['path_color'])
    for i in range(len(walk['path']) - 1):
        glVertex2f(*walk['path'][i])
        glVertex2f(*walk['path'][i + 1])
    glEnd()

# Function to draw the current point
def draw_current_point(walk):
    glColor3f(*walk['color'])
    glPointSize(5.0)  # Point size without scaling
    glBegin(GL_POINTS)
    glVertex2f(walk['x'], walk['y'])
    glEnd()

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    for walker, walk in enumerate(walks):
        random_walk_step(walker, walk)
    
    # Update the midpoint position
    update_midpoint(midpoint, walks[0], walks[1])
    
    # Clear screen
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # Draw paths and current points for both walks
    for walk in walks:
        draw_path(walk)
        draw_current_point(walk)
    
    # Draw path and current point for the midpoint
    draw_path(midpoint)
    draw_current_point(midpoint)
    
    # Update display
    pygame.display.flip()
    pygame.time.wait(10)

pygame.quit()

The above code works (and the basis choice is represented as the random choice between positive and negative movement)
The problem is that each successive movement is not related to the previous one thus even though for each individual movement there is a correlation between how A and B moved overall there is no structure in the midpoint dot. We somehow need to make it clear that there is a relationship between A and B

Maybe coloring each move based on if the two particles had a correlated behaviour. if both go horizontal its blue, if both go vertical its red. otherwise its white. This doesnt take into account the basis choice though and we dont see correlations if they are HV and VH